In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

import shap



In [2]:
#Load your dataset (replace with the correct path to your Excel file)
df = pd.read_excel('/home/sruthi/conference/filtered_data_DLS_diameter.xlsx')


# Exclude Abs_max and FW_80M features
features_to_exclude = [ 'Name','DLS Diameter [nm]','Standard Deviation DLS [nm]']
X = df.drop(columns=features_to_exclude)

# Use DLS_diameter as the label
y = df['DLS Diameter [nm]']

# Convert DLS_diameter to binary classification
y = (y >= 50).astype(int)  # 1 for >= 50, 0 for < 50

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [3]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Using lazypredict to get several classifier model outputs

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyClassifier

In [ ]:
# Run LazyClassifier
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

# Display the results
print(models)

Doing classification but with LGBM classifier

In [6]:
# Load your dataset (replace with the correct path to your Excel file)
df = pd.read_excel('/home/sruthi/conference/filtered_data_DLS_diameter.xlsx')


# Exclude Abs_max and FW_80M features
features_to_exclude = [ 'Name','DLS Diameter [nm]','Standard Deviation DLS [nm]']
X = df.drop(columns=features_to_exclude)

# Use DLS_diameter as the label
y = df['DLS Diameter [nm]']

# Convert DLS_diameter to binary classification
y = (y >= 50).astype(int)  # 1 for >= 50, 0 for < 50

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [7]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=9999)

In [ ]:
from lightgbm import LGBMClassifier

# Initialize and train the LGBMClassifier
lgbm_model = lgbm_model = LGBMClassifier(random_state=9999)
lgbm_model.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import (
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    roc_auc_score,
)
# Evaluation Metrics for Training Data
y_train_pred = lgbm_model.predict(X_train)
y_train_pred_proba = lgbm_model.predict_proba(X_train)[:, 1]

# Calculate metrics for training data
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
train_roc_auc = roc_auc_score(y_train, y_train_pred_proba)

# Compute confusion matrix components for training data
train_conf_matrix = confusion_matrix(y_train, y_train_pred)
tn_train, fp_train, fn_train, tp_train = train_conf_matrix.ravel()

# Calculate specificity for training data
train_specificity = tn_train / (tn_train + fp_train)

# Print training metrics
print("Training Data Evaluation Metrics:")
print(f"Accuracy: {train_accuracy:.2f}")
print(f"Precision: {train_precision:.2f}")
print(f"Recall: {train_recall:.2f}")
print(f"F1-Score: {train_f1:.2f}")
print(f"ROC-AUC Score: {train_roc_auc:.2f}")
print(f"Specificity: {train_specificity:.2f}")

# Evaluation Metrics for Test Data
y_pred = lgbm_model.predict(X_test)
y_pred_proba = lgbm_model.predict_proba(X_test)[:, 1]

# Calculate metrics for test data
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Compute confusion matrix components for test data
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()

# Calculate specificity for test data
specificity = tn / (tn + fp)

# Print test metrics
print("\nTest Data Evaluation Metrics:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"ROC-AUC Score: {roc_auc:.2f}")
print(f"Specificity: {specificity:.2f}")

# Compare Training and Test Metrics
print("\nComparison of Training vs Test Metrics:")
print(f"Accuracy - Train: {train_accuracy:.2f}, Test: {accuracy:.2f}")
print(f"Precision - Train: {train_precision:.2f}, Test: {precision:.2f}")
print(f"Recall - Train: {train_recall:.2f}, Test: {recall:.2f}")
print(f"F1-Score - Train: {train_f1:.2f}, Test: {f1:.2f}")
print(f"ROC-AUC - Train: {train_roc_auc:.2f}, Test: {roc_auc:.2f}")
print(f"Specificity - Train: {train_specificity:.2f}, Test: {specificity:.2f}")


Feature importance from LGBMclassifier

In [ ]:
# Assuming lgbm_model is already trained, and X is a NumPy array, but we need original feature names.
# For example, use the original DataFrame `df` or `X_train` that you used before scaling
feature_names = df.drop(columns=['Name', 'DLS Diameter [nm]', 'Standard Deviation DLS [nm]']).columns

# Get feature importance based on split count
importance_split = lgbm_model.feature_importances_

# Combine into a DataFrame
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance (Split)': importance_split
})

# Sort and display feature importance
importance_df = importance_df.sort_values(by='Importance (Split)', ascending=False)
print(importance_df)

# Plot feature importance (Split)
lgb.plot_importance(lgbm_model, importance_type='split', max_num_features=10, figsize=(10, 6))
plt.title("Feature Importance - Split Count")
plt.show()

# Plot feature importance (Gain)
lgb.plot_importance(lgbm_model, importance_type='gain', max_num_features=10, figsize=(10, 6))
plt.title("Feature Importance - Gain")
plt.show()


In [ ]:
# Get feature names and their importances
feature_names = df.drop(columns=['Name', 'DLS Diameter [nm]', 'Standard Deviation DLS [nm]']).columns
importance_split = lgbm_model.feature_importances_

# Combine into a DataFrame
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance (Split)': importance_split
}).sort_values(by='Importance (Split)', ascending=False)

# Display the sorted importance values
print(importance_df)

# Custom plot for split importance with split values displayed
plt.figure(figsize=(10, 6))
y_values = importance_df['Feature'][:20][::-1]
x_values = importance_df['Importance (Split)'][:20][::-1]
plt.barh(y_values, x_values, color='skyblue')

# Annotate the bars with their importance values
for i, (value, name) in enumerate(zip(x_values, y_values)):
    plt.text(value + 2, i, str(value), va='center', fontsize=9, color='black')

plt.xlabel("Importance (Split)")
plt.title("Feature Importance - Split Count (with Values)")
plt.tight_layout()

# Save the split importance plot as a JPG
plt.savefig("feature_importance_split_with_values.jpg", format="jpg")
plt.show()
# Save the gain importance plot as a JPG
plt.savefig("feature_importance_gain_with_values.jpg", format="jpg")
plt.show()

Top 6 features from feature importance

In [ ]:
# Random state for reproducibility
rs = 99

# Load your dataset
df = pd.read_excel('/home/sruthi/conference/filtered_data_DLS_diameter.xlsx')

# Exclude irrelevant features from the dataset
features_to_exclude = ['Name', 'DLS Diameter [nm]', 'Standard Deviation DLS [nm]']
X_df = df.drop(columns=features_to_exclude)

# Strip leading and trailing spaces from column names
X_df.columns = X_df.columns.str.strip()

# Print out the columns to check for any discrepancies
print("Columns in the dataset:", X_df.columns)

# Use DLS_diameter as the label and convert it to binary classification (1 if >= 50, else 0)
y = df['DLS Diameter [nm]']
y = (y >= 50).astype(int)  # 1 for >= 50, 0 for < 50

# Top 6 features based on feature importance
top_6_features = [
    '$n_\mathrm{t}$(AA)/n(Au)', 'd$n$(AA)/d$t$ [µmol/min]', '$V$(Seeds) [µL]',
    '$c_\mathrm{f}$(AA) [mM]', 'd$V$(Au)/d$t$ [µL/min]', 'd$n$(Au)/d$t$ [µmol/min]'
]

# Ensure all features are present in the DataFrame
missing_features = [feature for feature in top_6_features if feature not in X_df.columns]
if missing_features:
    print("Missing features:", missing_features)
else:
    # Select only the top 6 features
    X_top_6_df = X_df[top_6_features]

    # Normalize the features using StandardScaler
    scaler = StandardScaler()
    X_top_6_scaled = scaler.fit_transform(X_top_6_df)

    # Split into training and testing sets with random_state=rs
    X_train, X_test, y_train, y_test = train_test_split(X_top_6_scaled, y, test_size=0.3, random_state=rs)

    # Initialize and train the LGBMClassifier
    lgbm_model = LGBMClassifier(random_state=rs)
    lgbm_model.fit(X_train, y_train)

    # Evaluation Metrics for Training Data
    y_train_pred = lgbm_model.predict(X_train)
    y_train_pred_proba = lgbm_model.predict_proba(X_train)[:, 1]

    # Calculate metrics for training data
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)
    train_roc_auc = roc_auc_score(y_train, y_train_pred_proba)

    # Compute confusion matrix components for training data
    train_conf_matrix = confusion_matrix(y_train, y_train_pred)
    tn_train, fp_train, fn_train, tp_train = train_conf_matrix.ravel()

    # Calculate specificity for training data
    train_specificity = tn_train / (tn_train + fp_train)

    # Print training metrics
    print("Training Data Evaluation Metrics:")
    print(f"Accuracy: {train_accuracy:.2f}")
    print(f"Precision: {train_precision:.2f}")
    print(f"Recall: {train_recall:.2f}")
    print(f"F1-Score: {train_f1:.2f}")
    print(f"ROC-AUC Score: {train_roc_auc:.2f}")
    print(f"Specificity: {train_specificity:.2f}")

    # Evaluation Metrics for Test Data
    y_pred = lgbm_model.predict(X_test)
    y_pred_proba = lgbm_model.predict_proba(X_test)[:, 1]

    # Calculate metrics for test data
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Compute confusion matrix components for test data
    conf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()

    # Calculate specificity for test data
    specificity = tn / (tn + fp)

    # Print test metrics
    print("\nTest Data Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"ROC-AUC Score: {roc_auc:.2f}")
    print(f"Specificity: {specificity:.2f}")

    # Compare Training and Test Metrics
    print("\nComparison of Training vs Test Metrics:")
    print(f"Accuracy - Train: {train_accuracy:.2f}, Test: {accuracy:.2f}")
    print(f"Precision - Train: {train_precision:.2f}, Test: {precision:.2f}")
    print(f"Recall - Train: {train_recall:.2f}, Test: {recall:.2f}")
    print(f"F1-Score - Train: {train_f1:.2f}, Test: {f1:.2f}")
    print(f"ROC-AUC - Train: {train_roc_auc:.2f}, Test: {roc_auc:.2f}")
    print(f"Specificity - Train: {train_specificity:.2f}, Test: {specificity:.2f}")

TOP 3 features from feature importance

In [ ]:
rs = 99
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from lightgbm import LGBMClassifier

# Load your dataset
df = pd.read_excel('/home/sruthi/conference/filtered_data_DLS_diameter.xlsx')

# Exclude features from the dataset
features_to_exclude = ['Name', 'DLS Diameter [nm]', 'Standard Deviation DLS [nm]']
X_df = df.drop(columns=features_to_exclude)

# Strip leading and trailing spaces from column names
X_df.columns = X_df.columns.str.strip()

# Print out the columns to check for any discrepancies
print("Columns in the dataset:", X_df.columns)

# Use DLS_diameter as the label and convert it to binary classification (1 if >= 50, else 0)
y = df['DLS Diameter [nm]']
y = (y >= 50).astype(int)  # 1 for >= 50, 0 for < 50

# Top 6 features based on feature importance
top_6_features = [
    '$n_\mathrm{t}$(AA)/n(Au)', 'd$n$(AA)/d$t$ [µmol/min]', '$V$(Seeds) [µL]'
]

# Ensure all features are present in the DataFrame
missing_features = [feature for feature in top_6_features if feature not in X_df.columns]
if missing_features:
    print("Missing features:", missing_features)
else:
    # Select only the top 6 features
    X_top_6_df = X_df[top_6_features]

    # Normalize the features using StandardScaler
    scaler = StandardScaler()
    X_top_6_scaled = scaler.fit_transform(X_top_6_df)

    # Split into training and testing sets with random_state=10 for reproducibility
    X_train, X_test, y_train, y_test = train_test_split(X_top_6_scaled, y, test_size=0.3, random_state=rs)

    # Initialize and train the LGBMClassifier
    lgbm_model = LGBMClassifier(random_state=rs)
    lgbm_model.fit(X_train, y_train)

    # Evaluation Metrics for Training Data
    y_train_pred = lgbm_model.predict(X_train)
    y_train_pred_proba = lgbm_model.predict_proba(X_train)[:, 1]

    # Calculate metrics for training data
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)
    train_roc_auc = roc_auc_score(y_train, y_train_pred_proba)

    # Compute confusion matrix components for training data
    train_conf_matrix = confusion_matrix(y_train, y_train_pred)
    tn_train, fp_train, fn_train, tp_train = train_conf_matrix.ravel()

    # Calculate specificity for training data
    train_specificity = tn_train / (tn_train + fp_train)

    # Print training metrics
    print("Training Data Evaluation Metrics:")
    print(f"Accuracy: {train_accuracy:.2f}")
    print(f"Precision: {train_precision:.2f}")
    print(f"Recall: {train_recall:.2f}")
    print(f"F1-Score: {train_f1:.2f}")
    print(f"ROC-AUC Score: {train_roc_auc:.2f}")
    print(f"Specificity: {train_specificity:.2f}")

    # Evaluation Metrics for Test Data
    y_pred = lgbm_model.predict(X_test)
    y_pred_proba = lgbm_model.predict_proba(X_test)[:, 1]

    # Calculate metrics for test data
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Compute confusion matrix components for test data
    conf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()

    # Calculate specificity for test data
    specificity = tn / (tn + fp)

    # Print test metrics
    print("\nTest Data Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"ROC-AUC Score: {roc_auc:.2f}")
    print(f"Specificity: {specificity:.2f}")

    # Compare Training and Test Metrics
    print("\nComparison of Training vs Test Metrics:")
    print(f"Accuracy - Train: {train_accuracy:.2f}, Test: {accuracy:.2f}")
    print(f"Precision - Train: {train_precision:.2f}, Test: {precision:.2f}")
    print(f"Recall - Train: {train_recall:.2f}, Test: {recall:.2f}")
    print(f"F1-Score - Train: {train_f1:.2f}, Test: {f1:.2f}")
    print(f"ROC-AUC - Train: {train_roc_auc:.2f}, Test: {roc_auc:.2f}")
    print(f"Specificity - Train: {train_specificity:.2f}, Test: {specificity:.2f}")

SHAP analysis

In [ ]:
# SHAP analysis
explainer = shap.Explainer(lgbm_model, X_train)
shap_values = explainer(X_train)

# SHAP summary plot with feature names from the dataset
shap.summary_plot(shap_values, X_train, feature_names=df.drop(columns=features_to_exclude).columns)

# Other SHAP plots
shap.summary_plot(shap_values, X_train, feature_names=df.drop(columns=features_to_exclude).columns, plot_type='bar')
#shap.summary_plot(shap_values, X_train, feature_names=df.drop(columns=features_to_exclude).columns, plot_type='dot')

# Save the plots as JPG
plt.savefig("feature_importance_split_with_values.jpg", format="jpg")
plt.show()
plt.savefig("feature_importance_gain_with_values.jpg", format="jpg")
plt.show()

top 5 features from SHAP analysis

In [ ]:
rs = 99
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from lightgbm import LGBMClassifier

# Load your dataset
df = pd.read_excel('/home/sruthi/conference/filtered_data_DLS_diameter.xlsx')

# Exclude features from the dataset
features_to_exclude = ['Name', 'DLS Diameter [nm]', 'Standard Deviation DLS [nm]']
X_df = df.drop(columns=features_to_exclude)

# Strip leading and trailing spaces from column names
X_df.columns = X_df.columns.str.strip()

# Print out the columns to check for any discrepancies
print("Columns in the dataset:", X_df.columns)

# Use DLS_diameter as the label and convert it to binary classification (1 if >= 50, else 0)
y = df['DLS Diameter [nm]']
y = (y >= 50).astype(int)  # 1 for >= 50, 0 for < 50

# Top 5 features based on SHAP analysis
top_5_features_SHAP = [
    '$n_\mathrm{t}$(AA)/n(Au)', 'd$n$(AA)/d$t$ [µmol/min]', '$V$(Seeds) [µL]',
    '$c_\mathrm{f}$(AA) [mM]', '$c_\mathrm{e}$(AA) [mM]'
]

# Ensure all features are present in the DataFrame
missing_features = [feature for feature in top_5_features_SHAP if feature not in X_df.columns]
if missing_features:
    print("Missing features:", missing_features)
else:
    # Select only the top 5 features
    X_top_5_df = X_df[top_5_features_SHAP]

    # Normalize the features using StandardScaler
    scaler = StandardScaler()
    X_top_5_scaled = scaler.fit_transform(X_top_5_df)

    # Split into training and testing sets with random_state=rs
    X_train, X_test, y_train, y_test = train_test_split(X_top_5_scaled, y, test_size=0.3, random_state=rs)

    # Initialize and train the LGBMClassifier
    lgbm_model = LGBMClassifier(random_state=rs)
    lgbm_model.fit(X_train, y_train)

    # Evaluation Metrics for Training Data
    y_train_pred = lgbm_model.predict(X_train)
    y_train_pred_proba = lgbm_model.predict_proba(X_train)[:, 1]

    # Calculate metrics for training data
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_precision = precision_score(y_train, y_train_pred)
    train_recall = recall_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred)
    train_roc_auc = roc_auc_score(y_train, y_train_pred_proba)

    # Compute confusion matrix components for training data
    train_conf_matrix = confusion_matrix(y_train, y_train_pred)
    tn_train, fp_train, fn_train, tp_train = train_conf_matrix.ravel()

    # Calculate specificity for training data
    train_specificity = tn_train / (tn_train + fp_train)

    # Print training metrics
    print("Training Data Evaluation Metrics:")
    print(f"Accuracy: {train_accuracy:.2f}")
    print(f"Precision: {train_precision:.2f}")
    print(f"Recall: {train_recall:.2f}")
    print(f"F1-Score: {train_f1:.2f}")
    print(f"ROC-AUC Score: {train_roc_auc:.2f}")
    print(f"Specificity: {train_specificity:.2f}")

    # Evaluation Metrics for Test Data
    y_pred = lgbm_model.predict(X_test)
    y_pred_proba = lgbm_model.predict_proba(X_test)[:, 1]

    # Calculate metrics for test data
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Compute confusion matrix components for test data
    conf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()

    # Calculate specificity for test data
    specificity = tn / (tn + fp)

    # Print test metrics
    print("\nTest Data Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"ROC-AUC Score: {roc_auc:.2f}")
    print(f"Specificity: {specificity:.2f}")

    # Compare Training and Test Metrics
    print("\nComparison of Training vs Test Metrics:")
    print(f"Accuracy - Train: {train_accuracy:.2f}, Test: {accuracy:.2f}")
    print(f"Precision - Train: {train_precision:.2f}, Test: {precision:.2f}")
    print(f"Recall - Train: {train_recall:.2f}, Test: {recall:.2f}")
    print(f"F1-Score - Train: {train_f1:.2f}, Test: {f1:.2f}")
    print(f"ROC-AUC - Train: {train_roc_auc:.2f}, Test: {roc_auc:.2f}")
    print(f"Specificity - Train: {train_specificity:.2f}, Test: {specificity:.2f}")